In [2]:
"Hello morning! I'm a Python script."

"Hello morning! I'm a Python script."

In [3]:
# unicode of "h" is 104
ord("h")

104

In [13]:
list("Hello morning! Hello chen, and Hello Chen Luo.".encode("utf-8"))
text = "Hello morning! Hello morning Chen, and Hello Chen."
tokens = text.encode("utf-8")
tokens = list(map(int, tokens)) # convert bytes to int in range 0-255 for convenience
print(len(tokens))
print(len(text))

50
50


In [14]:
def get_stats(ids):
    counts = {}
    # zip function is used to iterate over two lists simultaneously
    for pair in zip(ids, ids[1:]): # iterate over all pairs of consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(tokens)
print(stats)

{(72, 101): 3, (101, 108): 3, (108, 108): 3, (108, 111): 3, (111, 32): 3, (32, 109): 2, (109, 111): 2, (111, 114): 2, (114, 110): 2, (110, 105): 2, (105, 110): 2, (110, 103): 2, (103, 33): 1, (33, 32): 1, (32, 72): 2, (103, 32): 1, (32, 67): 2, (67, 104): 2, (104, 101): 2, (101, 110): 2, (110, 44): 1, (44, 32): 1, (32, 97): 1, (97, 110): 1, (110, 100): 1, (100, 32): 1, (110, 46): 1}


In [18]:
top_pair = max(stats, key=stats.get)
top_pair

(72, 101)

In [20]:
def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids
print(tokens)
print(merge(tokens, top_pair, 256))

[72, 101, 108, 108, 111, 32, 109, 111, 114, 110, 105, 110, 103, 33, 32, 72, 101, 108, 108, 111, 32, 109, 111, 114, 110, 105, 110, 103, 32, 67, 104, 101, 110, 44, 32, 97, 110, 100, 32, 72, 101, 108, 108, 111, 32, 67, 104, 101, 110, 46]
[256, 108, 108, 111, 32, 109, 111, 114, 110, 105, 110, 103, 33, 32, 256, 108, 108, 111, 32, 109, 111, 114, 110, 105, 110, 103, 32, 67, 104, 101, 110, 44, 32, 97, 110, 100, 32, 256, 108, 108, 111, 32, 67, 104, 101, 110, 46]


vocab_size = 262
num_merges = vocab_size - 256
ids = list(tokens)

merges = {} # (int, int) -> int

for i in range(num_merges):
    stats = get_stats(ids)
    top_pair = max(stats, key=stats.get)
    idx = 256 + i
    ids = merge(ids, top_pair, idx)
    merges[top_pair] = idx
    print("merging pair", top_pair, "into", idx)

In [23]:
print(len(tokens))
print (len(ids))

50
33


In [39]:
vocab = {idx: bytes([idx]) for idx in range(256)}
print(vocab)
for pair, idx in merges.items():
    vocab[idx] = vocab[pair[0]] + vocab[pair[1]] # concatenate the two bytes

print(vocab)


def decode(ids):
    # given ids, return the text
    tokens = b"".join(vocab[i] for i in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text

print(decode([257, 261]))

{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: b'V', 87: b'W', 88: b'X', 89: b'Y', 90: b'Z', 91: b'[',

In [54]:
print(vocab)
print(merges)


def encode(text):
    # given a string, return list of integers (the tokens)
    tokens = text.encode("utf-8")
    print ("utf-8", tokens)
    tokens = list(map(int, tokens))
    print(tokens)
    while True:
        has_pair = False
        for pair in zip(tokens, tokens[1:]):
            if pair in merges:
                tokens = merge(tokens, pair, merges[pair])
                # print("Inter Stage: ", tokens)
                has_pair = True
                # break
        if not has_pair:
            break
        
    return tokens

print(encode("Hello morning! Hello morning Chen, and Hello Chen."))

{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: b'V', 87: b'W', 88: b'X', 89: b'Y', 90: b'Z', 91: b'[',

In [55]:
print(decode([261, 111, 114, 110, 105, 110, 103, 33, 32, 261, 111, 114, 110, 105, 110, 103, 32, 67, 104, 101, 110, 44, 32, 97, 110, 100, 32, 260, 67, 104, 101, 110, 46]))
print(vocab)

Hello morning! Hello morning Chen, and Hello Chen.
{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: b'V'

In [56]:
valtext = "good morning chen, I am chen luo."
vatext2 = decode(encode(valtext))
print(valtext == vatext2)

utf-8 b'good morning chen, I am chen luo.'
[103, 111, 111, 100, 32, 109, 111, 114, 110, 105, 110, 103, 32, 99, 104, 101, 110, 44, 32, 73, 32, 97, 109, 32, 99, 104, 101, 110, 32, 108, 117, 111, 46]
True
